In [213]:
import numpy as np;
import pandas as pd;

In [214]:
df=pd.read_csv("loan_approval_dataset (1).csv")

In [215]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LN00001,Male,No,2,Not Graduate,No,20354,8912,179,240,1,Rural,Y
1,LN00002,Female,Yes,3,Not Graduate,Yes,24190,4733,243,180,1,Urban,N
2,LN00003,Male,Yes,1,Not Graduate,Yes,8142,7081,365,120,1,Urban,Y
3,LN00004,Male,Yes,3,Not Graduate,Yes,21203,8690,193,120,1,Urban,N
4,LN00005,Male,Yes,3,Graduate,No,11094,7117,65,360,1,Urban,Y


In [216]:
df.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [217]:
print(df["Education"].value_counts())

print()
print(df["Self_Employed"].value_counts())

print()
print(df["Property_Area"].value_counts())



Education
Not Graduate    524
Graduate        476
Name: count, dtype: int64

Self_Employed
Yes    511
No     489
Name: count, dtype: int64

Property_Area
Semiurban    346
Urban        342
Rural        312
Name: count, dtype: int64


In [218]:
df=df.drop(["Loan_ID","Gender"],axis=1)

In [219]:
df["Loan_Status"]=df["Loan_Status"].map({"Y":1,"N":0})

In [220]:
df.head()

,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,No,2,Not Graduate,No,20354,8912,179,240,1,Rural,1
1,Yes,3,Not Graduate,Yes,24190,4733,243,180,1,Urban,0
2,Yes,1,Not Graduate,Yes,8142,7081,365,120,1,Urban,1
3,Yes,3,Not Graduate,Yes,21203,8690,193,120,1,Urban,0
4,Yes,3,Graduate,No,11094,7117,65,360,1,Urban,1


In [221]:
# Existing imports
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer

# Additional imports for imbalanced data handling
from imblearn.pipeline import Pipeline as ImbPipeline  # Use this pipeline if SMOTE is included
from imblearn.over_sampling import SMOTE


In [222]:
x,y=df.drop("Loan_Status",axis=1),df["Loan_Status"]

In [223]:
x.shape,y.shape

((1000, 10), (1000,))

In [224]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [225]:
print("X --\n",x.isnull().sum())
print("Y --",y.isnull().sum())

X --
 Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
dtype: int64
Y -- 0


In [226]:
x.head()

,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,No,2,Not Graduate,No,20354,8912,179,240,1,Rural
1,Yes,3,Not Graduate,Yes,24190,4733,243,180,1,Urban
2,Yes,1,Not Graduate,Yes,8142,7081,365,120,1,Urban
3,Yes,3,Not Graduate,Yes,21203,8690,193,120,1,Urban
4,Yes,3,Graduate,No,11094,7117,65,360,1,Urban


In [227]:
Transformer1=ColumnTransformer(transformers=[
    ("ordinal_cols",OrdinalEncoder(categories=[["No","Yes"],["Not Graduate","Graduate"]]),["Married","Education"]),

],remainder="passthrough")

In [228]:
Transformer2=ColumnTransformer(transformers=[
    ("onehot_cols",OneHotEncoder(sparse_output=False ,drop="first"),[3,9]),
],remainder="passthrough")

In [280]:
from imblearn.over_sampling import SMOTE;
from imblearn.under_sampling import RandomUnderSampler;

In [281]:
Transformer3=RandomUnderSampler(random_state=42)

In [282]:
Transformer4=RandomForestClassifier();

In [283]:
pipe = ImbPipeline([
    ("Transformer1", Transformer1),
    ("Transformer2", Transformer2),
    ("Transformer3",Transformer3),   
    ("Transformer4", Transformer4)
])

In [284]:
from sklearn import set_config;

set_config(display="diagram")

In [285]:
pipe

Pipeline(steps=[('Transformer1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ordinal_cols',
                                                  OrdinalEncoder(categories=[['No',
                                                                              'Yes'],
                                                                             ['Not '
                                                                              'Graduate',
                                                                              'Graduate']]),
                                                  ['Married', 'Education'])])),
                ('Transformer2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehot_cols',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  [3, 9])])),
                ('Transformer3', RandomUnderSampler(random_state=42)),
                ('Transformer4', RandomForestClassifier())])

In [286]:
x_train=pd.DataFrame(x_train,columns=x.columns)

In [287]:
x_train

,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
29,Yes,3,Graduate,No,23162,5618,220,120,1,Semiurban
535,No,2,Graduate,Yes,22954,2696,468,120,0,Semiurban
695,No,1,Graduate,Yes,3481,4006,542,240,1,Rural
557,Yes,2,Graduate,No,18964,8924,515,120,1,Semiurban
836,No,1,Graduate,No,20485,9427,147,180,1,Rural
...,...,...,...,...,...,...,...,...,...,...
106,Yes,2,Not Graduate,No,6727,2890,343,180,1,Urban
270,No,0,Graduate,No,4158,6837,230,180,1,Urban
860,No,2,Not Graduate,No,6164,8790,177,120,1,Urban
435,No,3,Graduate,Yes,23542,226,58,360,1,Urban


In [288]:
import pandas as pd

# If X is your original DataFrame before train-test split
x_train = pd.DataFrame(x_train, columns=x.columns)
x_test = pd.DataFrame(x_test, columns=x.columns)



In [289]:
pipe.fit(x_train,y_train)

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\compose\_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('Transformer1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ordinal_cols',
                                                  OrdinalEncoder(categories=[['No',
                                                                              'Yes'],
                                                                             ['Not '
                                                                              'Graduate',
                                                                              'Graduate']]),
                                                  ['Married', 'Education'])])),
                ('Transformer2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehot_cols',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  [3, 9])])),
                ('Transformer3', RandomUnderSampler(random_state=42)),
                ('Transformer4', RandomForestClassifier())])

In [290]:
y_pred=pipe.predict(x_test)

In [291]:
acc=accuracy_score(y_test,y_pred)

In [292]:
print("accuracy is --> ",acc)

accuracy is -->  0.515


In [293]:
f1_score_test=f1_score(y_test,y_pred)

In [294]:
f1_score_test

0.5726872246696035

In [307]:
y_pred

array([1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 1])

In [295]:
y_pred_train=pipe.predict(x_train)
acc2=accuracy_score(y_train,y_pred_train)
print("trainig accuracy is --> ",acc2)
#Beacuse my model is overfit --- tning is needed ;

trainig accuracy is -->  0.79


Because of my model overfit tunning is needed


In [296]:
params = {
    "Transformer4__n_estimators": [50, 100, 200],
    "Transformer4__max_depth": [4, 6, 7],
    "Transformer4__min_samples_split": [7, 10, 12],
    "Transformer4__min_samples_leaf": [2,3,6],
}

grid=GridSearchCV(pipe,params,cv=4,scoring="accuracy")
grid.fit(x_train,y_train)





c:\ProgramData\anaconda3\Lib\site-packages\sklearn\compose\_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('Transformer1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ordinal_cols',
                                                                         OrdinalEncoder(categories=[['No',
                                                                                                     'Yes'],
                                                                                                    ['Not '
                                                                                                     'Graduate',
                                                                                                     'Graduate']]),
                                                                         ['Married',
                                                                          'Education'])])),
                                       ('Transformer2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('onehot_cols',
                                                                         OneHotEncoder(drop='first',
                                                                                       sparse_output=False),
                                                                         [3,
                                                                          9])])),
                                       ('Transformer3',
                                        RandomUnderSampler(random_state=42)),
                                       ('Transformer4',
                                        RandomForestClassifier())]),
             param_grid={'Transformer4__max_depth': [4, 6, 7],
                         'Transformer4__min_samples_leaf': [2, 3, 6],
                         'Transformer4__min_samples_split': [7, 10, 12],
                         'Transformer4__n_estimators': [50, 100, 200]},
             scoring='accuracy')

In [297]:
grid.best_score_

np.float64(0.5175)

In [298]:
grid.best_params_

{'Transformer4__max_depth': 4,
 'Transformer4__min_samples_leaf': 3,
 'Transformer4__min_samples_split': 10,
 'Transformer4__n_estimators': 50}

In [299]:
predict_=grid.predict(x_test)

In [300]:
y_train.value_counts()

Loan_Status
1    543
0    257
Name: count, dtype: int64

In [301]:
acc3=accuracy_score(y_test,predict_)

print(acc3)

0.57


In [302]:
from sklearn.metrics import f1_score

In [303]:
r2=f1_score(y_test,predict_)
print("accuracy",r2)

accuracy 0.6692307692307692


In [304]:
predict_

array([1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       0, 1])

In [305]:
import pickle;
pickle.dump(grid,open("model.pkl","wb"))

In [306]:
import sklearn
print(sklearn.__version__)

1.6.1


In [323]:
dataframe=pd.DataFrame([["Yes",2,"Not Graduate","No",18499,2294,178,240,0,"Urban"]],columns=x_train.columns)

practice_predict=grid.predict(dataframe)

In [324]:
print(practice_predict)

[1]
